In [1]:
import re, os, json

In [47]:
# 首先找一个
path = "/Users/yangxinyi/Library/Mobile Documents/com~apple~CloudDocs/100-CodeEngine/350-数据分析完善案例 - 高价值/20200209 - 基于欺诈检测的综合财务数据集预测金融支付服务中的欺诈行为/Kaggle源文件_ Predicting Fraud in Financial Payment Services_副本.ipynb"

In [48]:
with open(path, 'r', encoding='utf-8') as file:
    notebook = json.load(file)

In [49]:
notebook

{'cells': [{'cell_type': 'markdown',
   'metadata': {'_cell_guid': 'c7b45d97-d213-4bff-a39a-fb973be3d1c2',
    '_uuid': '40baadcf98e87b9fa263ec2c80804a6298d6472e'},
   'source': ["This dataset is presently only one of four on Kaggle with information on the rising risk of digital financial fraud, emphasizing the difficulty in obtaining such data. The main technical challenge it poses to predicting fraud is the highly imbalanced distribution between positive and negative classes in 6 million rows of data. Another stumbling block to the utility of this data stems from the possible  discrepancies in its description <a href='https://www.kaggle.com/ntnu-testimon/paysim1/discussion/35004'>[1]</a>, <a href='https://www.kaggle.com/lightcc/money-doesn-t-add-up/'>[2]</a>, <a href='https://www.kaggle.com/ntnu-testimon/paysim1/discussion/32786'>[3]</a>. The goal of this analysis is to solve both these issues by a detailed data exploration and cleaning followed by choosing a suitable machine-learnin

In [53]:
notebook

{'cells': [{'cell_type': 'markdown',
   'metadata': {'_cell_guid': 'c7b45d97-d213-4bff-a39a-fb973be3d1c2',
    '_uuid': '40baadcf98e87b9fa263ec2c80804a6298d6472e'},
   'source': ["This dataset is presently only one of four on Kaggle with information on the rising risk of digital financial fraud, emphasizing the difficulty in obtaining such data. The main technical challenge it poses to predicting fraud is the highly imbalanced distribution between positive and negative classes in 6 million rows of data. Another stumbling block to the utility of this data stems from the possible  discrepancies in its description <a href='https://www.kaggle.com/ntnu-testimon/paysim1/discussion/35004'>[1]</a>, <a href='https://www.kaggle.com/lightcc/money-doesn-t-add-up/'>[2]</a>, <a href='https://www.kaggle.com/ntnu-testimon/paysim1/discussion/32786'>[3]</a>. The goal of this analysis is to solve both these issues by a detailed data exploration and cleaning followed by choosing a suitable machine-learnin

In [50]:
py_code = ""
id_cell = 0
for cell in notebook['cells']:
    if cell['cell_type'] == 'code':
        
        # 将代码转换成字符串        
        code = ''.join(cell['source'])
        if code:
            # 检测并处理魔法命令
            code_lines = code.splitlines()
            for line in code_lines:
                if re.match(r'^\s*%', line):  # 如果该行是魔法命令
                    # 将魔法命令转换为注释，或者可以选择跳过这一行
                    py_code += '# ' + line + '\n'
                else:
                    py_code += line + '\n'
        py_code += "# Ú" + "\n" 
        id_cell += 1
    

In [51]:
print(py_code)

import pandas as pd
import numpy as np
# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz
# Ú
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Ú
df = pd.read_csv('../input/PS_20174392719_1491204439457_log.csv')
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})
print(df.head())
# Ú
df.isnull().values.any()
# Ú
print('\n The types of fraudulent transactions are {}'.format(\
list(df.loc[df.isFraud == 1].type.drop_duplicates().values))) # only 'CASH_OUT' 
                                                             #

In [52]:
# 将 py_code 处理回去
id_cell = 0
py_code_split_cell = py_code.split("# Ú\n")
for cell in notebook['cells']:
    if cell['cell_type'] == 'code':
        # 将py_code处理回去
        cell_code = py_code_split_cell[id_cell].removesuffix("\n").splitlines(keepends=True)
        cell['source'] = cell_code
        id_cell += 1

In [55]:
new_path = "/Users/yangxinyi/Library/Mobile Documents/com~apple~CloudDocs/100-CodeEngine/350-数据分析完善案例 - 高价值/20200209 - 基于欺诈检测的综合财务数据集预测金融支付服务中的欺诈行为/Kaggle源文件_ Predicting Fraud in Financial Payment Services_转换.ipynb"

In [56]:
# 保存
with open(new_path, "w") as f:
    json.dump(notebook, f, ensure_ascii=False, indent=4)

In [2]:
def ipynb转换成py(notebook):
    py_code = ""
    for cell in notebook['cells']:
        if cell['cell_type'] == 'code':
            # 将代码转换成字符串
            code = ''.join(cell['source'])

            # 检测并处理魔法命令
            code_lines = code.splitlines()
            for line in code_lines:
                if re.match(r'^\s*%', line):  # 如果该行是魔法命令
                    # 将魔法命令转换为注释，或者可以选择跳过这一行
                    py_code += '# ' + line + '\n'
                else:
                    py_code += line + '\n'

        elif cell['cell_type'] == 'markdown':
            # 将 Markdown 内容转换为注释
            comments = ''.join(cell['source'])
            comment_lines = comments.splitlines()
            for line in comment_lines:
                py_code += '# ' + line + '\n'

In [61]:
def 检查是否python英文变量(name):
    # 检查是否是字母或下划线开头
    if not name or not name[0].isalpha() and name[0] != '_':
        return False
    # 检查是否包含非ASCII字符
    if not name.isascii():
        return False
    # 检查剩余字符是否仅由字母、数字或下划线构成
    if not all(c.isalnum() or c == '_' for c in name):
        return False
    return True

In [64]:
检查是否python英文变量("1_abd_123")

False

In [60]:
"金".isalpha()

True